# Programming Project - Unit 2
*by Débora Azevedo, Eliseu Jayro, Francisco de Paiva and Igor Brandão*

**Goals**
The purpose of this project is explore the following:

- Access Health Graph API - Runkeeper content;
- Geolocation analysis and hypotheses should be explained in detail;

<hr>

# Global Imports section

Import the necessary libraries to handle 

- Geocoding;
- Maps;
- File input;
- Heatmap;
- Numpy library;
- Tqdm progress bar

In [ ]:
### Library necessary to run this IPython Notebook
!pip install geocoder
!pip install folium
!pip install tqdm
!pip install tabulate
!pip install pandas-datareader
!pip install requests

In [ ]:
# Import pandas
import pandas as pd

# Import json
import json

# Import google geocoder
import geocoder as gc

# Import numpy library
import numpy as np

# Import folium maps
import os
import folium
from folium import plugins

# Import tqdm progressing bar plugin
from tqdm import tqdm

<hr>

# I - Maps section

## Geolocation data import [optional]

In [ ]:
# Import the geolocation.csv data: data
geolocation_data = pd.read_csv("geolocation.csv", encoding = 'latin2')

### Coordinate groupping

The cell below perform the latitude/longitude reading and count all og them to know if there are repetitions and group *[latitude, longitude, count]* data into the coordinate array

In [ ]:
# =================================================================================
# Data adjusts
# =================================================================================

# Count the same latitude/longitude occurrences
geodata = geolocation_data.groupby(['longitude', 'latitude']).size().reset_index().rename(columns={0:'count'})

# =================================================================================
# Add the coordinates
# =================================================================================

# Initialize the coordinates array
coordinates = []

# Add the coordinates to the coordinate
for i in tqdm(range(len(geodata))):
    # eliminate items with'nan' element
    if all(~np.isnan([geodata.loc[geodata.index[i],'latitude'], geodata.loc[geodata.index[i],'longitude']])):
        coordinates.append([geodata.loc[geodata.index[i],'latitude'], geodata.loc[geodata.index[i],'longitude'], 
                            geodata.loc[geodata.index[i],'count']])

# Display the imported coordinates
coordinates

## Add the coordinates to the maps [required]

In [29]:
# =================================================================================
# Map settings
# =================================================================================

# Set map center and zoom level
mapc = [-5.788, -35.202]
zoom = 11

# Wind rose
url = ('https://raw.githubusercontent.com/SECOORA/static_assets/'
       'master/maps/img/rose.png')

# Create HeatMap instance
htMap = folium.Map(location=mapc, zoom_start=zoom, tiles='OpenStreetMap')

# Create marker map instance
mkMap = folium.Map(location=mapc, zoom_start=zoom, tiles='OpenStreetMap')

# Create circle marker map instance
circMap = folium.Map(location=mapc, zoom_start=zoom, tiles='OpenStreetMap')

# Chunk size
chunksize = 10 ** 8 # can be changed

# For item i in a range that is a length of l,
for i in range(0, len(coordinates), chunksize):
    # Append the coordinates to the Heat Map
    plugins.HeatMap(coordinates[i:i+chunksize]).add_to(htMap)
    plugins.FloatImage(url, bottom=5, left=75).add_to(htMap)
    
    # Append the coordinates to Marker Map
    plugins.MarkerCluster(coordinates[i:i+chunksize]).add_to(mkMap)
    plugins.FloatImage(url, bottom=5, left=75).add_to(htMap)

## 1) Heat Map

In [ ]:
# Print the heatMap
htMap

## 2) Cluster Map

In [ ]:
# Print the cluster map
mkMap

## 3) With Natal Neighborhoods

In [ ]:
# import geojson file about natal neighborhoods
natal_neigh = os.path.join('geojson', 'natal.geojson')

# load the data and use 'UTF-8'encoding
geo_json_natal = json.load(open(natal_neigh,encoding='UTF-8'))

In [ ]:
# print the keys of the dictionary
print(geo_json_natal.keys())
# print the list of features (neighborhoods)
geo_json_natal['features']

In [ ]:
neighborhood = []
# list all neighborhoods
for neigh in geo_json_natal['features']:
        neighborhood.append(neigh['properties']['name'])

### 3.1) Heatmap with Natal neighborhoods

In [ ]:
# Create new map instance
htMapNeighborhood = htMap

# Configure geojson layer
folium.GeoJson(geo_json_natal).add_to(htMapNeighborhood)

# print the map
htMapNeighborhood

## 4) Choropleth map